In [3]:
# pip install PyPDF2

In [4]:
# pip install pyttsx3

In [8]:
import tkinter as tk
from tkinter import filedialog
import pyttsx3
from PyPDF2 import PdfReader
import threading

class PDFReaderApp:
    def __init__(self, root):
        self.root = root
        self.root.title("PDF Reader")

        # Initialize the pyttsx3 speaker engine
        self.speaker = pyttsx3.init()
        self.speaker.setProperty('rate', 150)  # Default speech rate
        self.speaker.setProperty('volume', 1)  # Set the volume to maximum

        # Get available voices
        self.voices = self.speaker.getProperty('voices')

        self.file_path = None
        self.is_reading = False
        self.text_to_read = ""

        # Create the GUI elements
        self.start_button = tk.Button(self.root, text="Start", command=self.start_reading)
        self.start_button.pack(pady=10)

        self.stop_button = tk.Button(self.root, text="Stop", command=self.stop_reading)
        self.stop_button.pack(pady=10)

        self.select_file_button = tk.Button(self.root, text="Select PDF", command=self.select_pdf)
        self.select_file_button.pack(pady=10)

        # Speed slider (range 50-300 for speech rate)
        self.speed_label = tk.Label(self.root, text="Adjust Speed")
        self.speed_label.pack(pady=5)

        self.speed_slider = tk.Scale(self.root, from_=50, to=300, orient='horizontal', length=200, label="Speech Speed")
        self.speed_slider.set(150)  # Default speech rate
        self.speed_slider.pack(pady=10)

        # Voice selection
        self.voice_label = tk.Label(self.root, text="Select Voice")
        self.voice_label.pack(pady=5)

        self.voice_var = tk.StringVar()
        self.voice_var.set("male")  # Default voice is male

        self.male_voice_radio = tk.Radiobutton(self.root, text="Male", variable=self.voice_var, value="male")
        self.male_voice_radio.pack(pady=5)

        self.female_voice_radio = tk.Radiobutton(self.root, text="Female", variable=self.voice_var, value="female")
        self.female_voice_radio.pack(pady=5)

    def select_pdf(self):
        """Open a file dialog to select a PDF."""
        file_path = filedialog.askopenfilename(
            title="Select a PDF file",
            filetypes=(("PDF files", "*.pdf"), ("All files", "*.*"))
        )

        if file_path:
            self.file_path = file_path
            print(f"Selected PDF: {self.file_path}")
        else:
            print("No file selected.")

    def extract_text_from_pdf(self):
        """Extract text from the selected PDF."""
        if not self.file_path:
            print("No file selected. Please select a PDF first.")
            return

        reader = PdfReader(self.file_path)
        text = ""
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            page_text = page.extract_text()
            if page_text:
                text += page_text
        return text

    def start_reading(self):
        """Start reading the PDF aloud."""
        if not self.file_path:
            print("No PDF selected! Please select a PDF file.")
            return

        if self.is_reading:
            print("Already reading the PDF.")
            return

        self.is_reading = True
        self.text_to_read = self.extract_text_from_pdf()

        if not self.text_to_read:
            print("No text extracted from the PDF.")
            return

        # Print extracted text for debugging
        print(f"Extracted Text:\n{self.text_to_read[:100]}...")  # Print first 100 characters for preview

        # Start the reading in a new thread
        reading_thread = threading.Thread(target=self.read_text)
        reading_thread.start()

    def read_text(self):
        """Run the text-to-speech process."""
        # Set the speech rate from the slider
        rate = self.speed_slider.get()
        self.speaker.setProperty('rate', rate)

        # Get the selected voice (male or female)
        voice_choice = self.voice_var.get()

        if voice_choice == "male":
            self.speaker.setProperty('voice', self.voices[0].id)  # Male voice (usually the first available voice)
        elif voice_choice == "female":
            self.speaker.setProperty('voice', self.voices[1].id)  # Female voice (usually the second available voice)

        # Speak the extracted text
        self.speaker.say(self.text_to_read)
        self.speaker.runAndWait()

        self.is_reading = False

    def stop_reading(self):
        """Stop reading the PDF aloud."""
        if self.is_reading:
            self.speaker.stop()
            self.is_reading = False
            print("Reading stopped.")
        else:
            print("No reading is in progress.")


# Create the Tkinter window and pass it to the PDFReaderApp
root = tk.Tk()
app = PDFReaderApp(root)

# Start the Tkinter mainloop
root.mainloop()


Selected PDF: C:/Users/nagen/Downloads/Lesson 1 - Introduction to AI and Machine Learning.pdf
Extracted Text:
©Simplilearn. All rights reserved.
Lesson 1: Introduction to Artificial Intelligence and         
Ma...
Reading stopped.
